# Final models

The purpose of this notebook is to collect "final versions" of the models we want to use for ensembling. These will all use the same data (though they can certainly omit columns if you want). The point is to go from zero-to-trained model in one notebook so we can make sure everything looks good.

In [9]:
import pandas as pd
import numpy as np

from utils.data_loader import Dataset

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, concatenate
from tensorflow.keras.layers.experimental import preprocessing

import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

## Data loading

Load the data here. Models below can augment it if need-be (for example the neural net needs it as a tensorflow dataset).

In [10]:
df = pd.read_csv('../data/Final Data/pct-diff-mlb-games.csv')  #Pct Diff Columns Only (Gives Highest Accuracy)
#df = pd.read_csv('../data/Final Data/diff-mlb-games.csv')    #Diff columns only
#df = pd.read_csv('../data/Final Data/full-diff-mlb-games.csv')    #All columns

train_df = df[df['Y'] <= 2015]
test_df = df[df['Y'] > 2015]

X_train = train_df.drop('home_win', axis=1)
y_train = train_df.home_win

X_test = test_df.drop('home_win', axis=1)
y_test = test_df.home_win

## XGBoost

In [ ]:
model = xgb.XGBClassifier()
xgbmodel = model.fit(X_train, y_train)

xgbpreds = xgbmodel.predict(X_test)

accuracy_score(y_test, xgbpreds)

In [ ]:
print(classification_report(y_test, xgbpreds))

### Feature Importance

In [ ]:
featuredf = pd.DataFrame(xgbmodel.feature_importances_, X_train.columns)   #This makes it in order
featuredf = featuredf.sort_values(by=0, ascending=False)

In [ ]:
plt.figure(figsize=(20, 8))

x = featuredf.index
y = featuredf[0]

plt.bar(x, y)
plt.xticks(rotation=90)
plt.title('Feature Importances', fontsize=20)
plt.ylabel('Weight', fontsize=14);

### Accuracy By Month

In [ ]:
def month_accuracy(model)
    monthly_acc = []
    months_list = np.sort(X_test['M'].unique())
    for months in months_list:
        test_month = test[test['M'] == months]

        X_test_month = test_month.drop('home_win', axis=1)
        y_test_month = test_month.home_win

        pred = model.predict(X_test_month)
        acc = accuracy_score(y_test_month, pred)

        monthly_acc.append(acc)

    monthly_acc = pd.DataFrame(monthly_acc)
    
    return monthly_acc

In [ ]:
xgb_month = monthly_accuracy(xgbmodel)

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(xgb_month.index, xgb_month[0]*100, alpha=0.75)
plt.ylabel('Accuracy', fontsize=16)
plt.xticks([0,1,2,3,4,5,6,7], ['March','April', 'May', 'June','July', 'August', 'September', 'October']);

## KNN

In [ ]:
nn_clf = KNeighborsClassifier()
nn_clf.fit(X_train, y_train)

nn_preds = nn_clf.predict(X_test)

accuracy_score(y_test, nn_preds)

In [ ]:
print(classification_report(y_test, nn_preds))

### Accuracy by Month

In [ ]:
nn_month = monthly_accuracy(nn_clf)

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(nn_month.index, nn_month[0]*100, alpha=0.75)
plt.ylabel('Accuracy', fontsize=16)
plt.xticks([0,1,2,3,4,5,6,7], ['March','April', 'May', 'June','July', 'August', 'September', 'October']);

## Neural net

### Make tensorflow dataset from data

In [16]:
def load_ds(df):
    cols_to_drop = ['home_pitcher', 'away_pitcher', 'date']
    cols_to_drop = list(set(df.columns).intersection(set(cols_to_drop)))
    df = df.drop(cols_to_drop, axis='columns')
    
    embedding_cols = ['home_team', 'away_team', 'Y', 'M']
    numeric_cols = list(set(df.columns) - set(embedding_cols))
    numeric_cols.remove('home_win')
    assert set(numeric_cols).intersection(set(embedding_cols)) == set()
    assert len(embedding_cols) + len(numeric_cols) + 1 == len(df.columns)
    
    for c in df.columns:
        if df[c].isin([-np.inf, np.inf]).sum() > 0:
            df[c] = df[c].replace([-np.inf, np.inf], None)
        if df[c].isna().sum() > 0:
            med = df[c].median()
            df[c] = df[c].fillna(med)
            
    le = LabelEncoder()
    df['away_team'] = le.fit_transform(df['away_team'])
    df['home_team'] = le.transform(df['home_team'])
    
    for c in embedding_cols:
        df[c] = df[c].astype(int)
        
    assert df.isna().sum().sum() == 0
    
    y = df.pop('home_win')
    y = y.astype(int)
    X = df
    
    tf_ds = tf.data.Dataset.from_tensor_slices((dict(X), y)).batch(128)
    return tf_ds, df, le, embedding_cols, numeric_cols

In [18]:
train_ds, train_df, le, embedding_cols, numeric_cols = load_ds(train_df)
test_ds, test_df, _, _, _ = load_ds(test_df)

### Prepare layers

In [28]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for our feature.
    normalizer = preprocessing.Normalization()

    # Prepare a Dataset that only yields our feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a StringLookup layer which will turn strings into integer indices
    if dtype == 'string':
        index = preprocessing.StringLookup(max_tokens=max_tokens)
    else:
        index = preprocessing.IntegerLookup(max_values=max_tokens)

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)
    
    if name == 'home_team':
        print('home_team')
        print(index.get_vocabulary())
        
    if name == 'away_team':
        print('away_team')
        print(index.get_vocabulary())

    # Create a Discretization for our integer indices.
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

    # Prepare a Dataset that only yields our feature.
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices.
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices. The lambda function captures the
    # layer so we can use them, or include them in the functional model later.
    return lambda feature: encoder(index(feature))

def prep_columns(dataset, embedding_dims=10, embedding_cols=None, numeric_cols=None):
    cont_inputs = []
    cat_inputs = []
    encoded_cont_features = []
    encoded_cat_features = []
    if isinstance(embedding_dims, int):
        embedding_dims = [embedding_dims] * len(embedding_cols)
    assert len(embedding_dims) == len(embedding_cols), 'embedding_dims must be an integer or a list with the same length as embedding_cols'

    # Numeric features.
    for header in numeric_cols:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, dataset)
        encoded_numeric_col = normalization_layer(numeric_col)
        encoded_numeric_col = tf.keras.layers.Dropout(0.1)(encoded_numeric_col)
        cont_inputs.append(numeric_col)
        encoded_cont_features.append(encoded_numeric_col)

    # Home and away teams
    for i, header in enumerate(['home_team', 'away_team']):
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
        encoded_categorical_col = tf.keras.layers.Embedding(30, embedding_dims[i], name=f'{header}_embedding')(categorical_col)
        encoded_categorical_col = tf.keras.layers.Flatten()(encoded_categorical_col)
        cat_inputs.append(categorical_col)
        encoded_cat_features.append(encoded_categorical_col)
        
    # Categorical features encoded as ints.
    encoded_embedding_cols = embedding_cols.copy()
    encoded_embedding_cols.remove('home_team')
    encoded_embedding_cols.remove('away_team')
    for i, header in enumerate(encoded_embedding_cols):
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
        encoding_layer = get_category_encoding_layer(header, dataset, 
                                                     dtype='int', 
                                                     max_tokens=20)
        encoded_categorical_col = encoding_layer(categorical_col)
        encoded_categorical_col = tf.keras.layers.Embedding(20, 5, name=f'{header}_embedding')(encoded_categorical_col)
        encoded_categorical_col = tf.keras.layers.Flatten()(encoded_categorical_col)
        cat_inputs.append(categorical_col)
        encoded_cat_features.append(encoded_categorical_col)
        
    all_inputs = cont_inputs + cat_inputs
        
    return all_inputs, encoded_cont_features, encoded_cat_features

In [29]:
all_inputs, encoded_cont_features, encoded_cat_features = prep_columns(train_ds,
                                                                       10,
                                                                       embedding_cols, 
                                                                       numeric_cols)

Numerical columns: ['pitcher_IP_pct_diff', 'team_FP_pct_diff', 'avg_pct_diff', 'team_W-L_pct_diff', 'WPA_pct_diff', 'win_pct_diff', 'ops_pct_diff', 'log_5', 'FP_pct_diff', 'team_WHIP_pct_diff', 'bayes_pct_diff', 'team_Rank_pct_diff', 'RA_pct_diff', 'pytha_pct_diff', 'RD_pct_diff', 'obp_pct_diff', 'R_pct_diff', 'team_ERA_pct_diff', 'slg_pct_diff', 'pitcher_WHIP_pct_diff', 'pitcher_ERA_pct_diff', 'Rank_pct_diff']
Embedding home_team and away_team
Categorical columns: ['Y', 'M']


### Model

In [63]:
x = concatenate(encoded_cat_features + encoded_cont_features)
x = Dense(32, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(32, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
output = Dense(1)(x)
del model
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [64]:
model.fit(train_ds, epochs=30, validation_data=test_ds)

Epoch 1/30
285/285 [==============================] - 4s 7ms/step - loss: 0.7246 - accuracy: 0.5544 - val_loss: 0.6648 - val_accuracy: 0.5947
Epoch 2/30
285/285 [==============================] - 1s 5ms/step - loss: 0.6638 - accuracy: 0.5801 - val_loss: 0.6616 - val_accuracy: 0.6071
Epoch 3/30
285/285 [==============================] - 2s 5ms/step - loss: 0.6491 - accuracy: 0.5883 - val_loss: 0.6578 - val_accuracy: 0.6054
Epoch 4/30
285/285 [==============================] - 2s 6ms/step - loss: 0.6455 - accuracy: 0.5915 - val_loss: 0.6561 - val_accuracy: 0.6055
Epoch 5/30
285/285 [==============================] - 1s 5ms/step - loss: 0.6444 - accuracy: 0.5957 - val_loss: 0.6560 - val_accuracy: 0.6005
Epoch 6/30
285/285 [==============================] - 2s 7ms/step - loss: 0.6407 - accuracy: 0.5966 - val_loss: 0.6527 - val_accuracy: 0.6003
Epoch 7/30
285/285 [==============================] - 1s 5ms/step - loss: 0.6390 - accuracy: 0.6024 - val_loss: 0.6531 - val_accuracy: 0.5989
Epoch 

In [65]:
loss, acc = model.evaluate(test_ds)
print(f'Test accuracy = {100*acc:.2f}%')

76/76 [==============================] - 0s 3ms/step - loss: 0.7174 - accuracy: 0.6007
Test accuracy = 60.07%
